In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Specify the input shape and number of classes (e.g., two classes for binary classification)
input_shape = (224, 224, 3)
num_classes = 2  # Modify based on your dataset

# a. Load in a pre-trained CNN model (e.g., MobileNetV2)
base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')

# b. Freeze parameters (weights) in model’s lower convolutional layers
for layer in base_model.layers:
    layer.trainable = False  # Freeze base layers

# c. Add custom classifier with several layers of trainable parameters
x = base_model.output
x = GlobalAveragePooling2D()(x)       # Global pooling layer
x = Dense(1024, activation='relu')(x) # Dense layer
x = Dropout(0.5)(x)                   # Dropout layer to reduce overfitting
x = Dense(512, activation='relu')(x)  # Additional dense layer
predictions = Dense(num_classes, activation='softmax')(x)  # Output layer for classification

# Final model with the custom classifier on top of the pre-trained base model
model = Model(inputs=base_model.input, outputs=predictions)

# d. Train classifier layers on training data available for task
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Data Preparation (use ImageDataGenerator for data augmentation)
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.15,
                                   width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.15, horizontal_flip=True, fill_mode="nearest")
val_datagen = ImageDataGenerator(rescale=1./255)

train_dir = "path/to/train"  # Specify path to training data
val_dir = "path/to/validation"  # Specify path to validation data

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224),
                                                    batch_size=32, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(224, 224),
                                                batch_size=32, class_mode='categorical')

# Train the model
history = model.fit(train_generator, validation_data=val_generator, epochs=10, verbose=2)

# e. Fine-tune hyperparameters and unfreeze more layers as needed
# Unfreeze some layers for fine-tuning
for layer in base_model.layers[-20:]:  # Adjust number of layers to unfreeze
    layer.trainable = True

# Recompile model with a lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model again with unfrozen layers
fine_tune_history = model.fit(train_generator, validation_data=val_generator, epochs=10, verbose=2)

# Display the training history
import matplotlib.pyplot as plt

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(fine_tune_history.history['accuracy'], label='Fine-tuned Training Accuracy')
plt.plot(fine_tune_history.history['val_accuracy'], label='Fine-tuned Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()
